# Piedras lunares - Toma de desiciones - Prediccion para misiones Artemis

En el siguiente analisis vamos a tratar de predecir la cantidad de material que debera traer las misiones [Artemis](https://www.nasa.gov/specials/artemis/) (proximamente misiones de la NASA) para poder complementar las piedras que pudieron ser traidas por las misiones Apolo.

Para eso primero vamos a importar Pandas para poder manejar los datos a futuro desde python.

In [ ]:
import pandas as pd

Debemos extraer datos de algun lado, para eso vamos a tomar una tabla con extension **CSV** para poder luego manipular sus datos con Pandas. Ese archivo se encuentra disponible abiertamente en el siguiente [Link](https://github.com/drguthals/learnwithdrg/tree/main/OverTheMoon/sample-return/data).

Una vez descaragdo lo guardaremos en la carpeta "data" para manejarlo desde ahi y poder leerlo con Pandas.

Para leer y manipular el archivo se usa el siguiente comando:

In [33]:
rock_samples=pd.read_csv('data/rocksamples.csv')

Ya tenemos a disposicion la lista de las piedras lunares, pero queremos ver como se muestra la lista, para eso usamos el comando que se ejecuta a continuacion

In [34]:
rock_samples.head(6)

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31
5,10008,Apollo11,Soil,Unsieved,89.0,5.75


In [35]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


### **Para tener en cuenta....**
El peso del cohete se suele medir en kilogramos, no en gramos. Por tanto, para manipular los datos originales, los pesos de las muestras se deben convertir a kilogramos.

In [36]:
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


En el anterior punto ademas de cambiar la unidad, notese el cambio del titulo de la columna.

Se va a crear un elemento dataframe con el nombre missions; será un resumen de los datos de cada una de las seis misiones Apolo en las que se han traído muestras.Ademas se creara una columna con el nombre Mission (una fila por mision Apolo).

Dentro de la funcion head colocamos 6 ya que sabemos que el archivo CSV contiene 6 misiones con sus datos.


In [37]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head(6)

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16
5,Apollo17


## **Suma total del peso de las muestras por misión**

Ahora puede agregar una columna nueva a missions para representar la suma de todas las muestras obtenidas durante esa misión.

In [38]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


## **Obtención de la diferencia de pesos entre misiones**

En el siguiente paso lo que vamos a hacer es sacar la diferencia de pesos entre las diferentes misiones para saber como fue aumentando o bajando la cantidad de material que se trajo de la luna. Para eso vamos a agregar una nueva columna que nos muestre la diferencia entre las misiones.

In [39]:
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


Como la primer fila de diferencia de pesos no puedo compararse ya que no tiene un valor previo nos da un valor **NaN**, para hacerlo mas "manejable" vamos a convertir este valor a 0 para que la columna solo maneje valores numericos.

In [40]:
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


Muy bien, hasta ahora pudimos manejar los datos de pesos de las piedras lunares, pero que hay del peso mismo del cohete y modulo lunar? es importante tener en cuenta estos datos ya que el peso de estos elementos determinara muchas cosas dentro de la mision, por eso a continuacion pasaremos a otra fase del analisis propuesto hasta ahora.

## **Agregando módulos de mando y lunar**

Con ayuda del [Archivo coordinado de datos de ciencia espacial de la NASA](https://nssdc.gsfc.nasa.gov/nmc/SpacecraftQuery.jsp), se ha obtenido información sobre cada uno de los módulos que se usa en cada misión.Lo que hacemos ahora es agregar 6 columnas (3 por modulos lunares y 3 por modulos de mando) como el detalles a continuacion:

Por cada modulo tendra las siguientes columnas:
    
    Nombre del módulo
    Masa del módulo
    Diferencia de la masa del módulo

los valores de **NaN** se convertiran a 0 para mantener numeros manejables en la tabla.



In [41]:
missions['Lunar module (LM)'] = {'Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)'}
missions['LM mass (kg)'] = {15103, 15235, 15264, 16430, 16445, 16456}
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = {'Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'}
missions['CM mass (kg)'] = {5560, 5609, 5758, 5875, 5840, 5960}
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Kitty Hawk (CM-110),5960,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Columbia (CSM-107),5609,-351.0
2,Apollo14,41.83363,7.49125,Challenger (LM-12),16456,1221.0,America (CM-114),5840,231.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Endeavor (CM-112),5875,35.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Yankee Clipper (CM-108),5560,-315.0
5,Apollo17,109.44402,16.98140,Intrepid (LM-6),15103,-1342.0,Casper (CM-113),5758,198.0


Podemos agregar el peso total de las misiones y la diferencia entre ellos.

In [42]:
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Kitty Hawk (CM-110),5960,0.0,21224,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Columbia (CSM-107),5609,-351.0,20844,-380.0
2,Apollo14,41.83363,7.49125,Challenger (LM-12),16456,1221.0,America (CM-114),5840,231.0,22296,1452.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Yankee Clipper (CM-108),5560,-315.0,22005,-300.0
5,Apollo17,109.44402,16.98140,Intrepid (LM-6),15103,-1342.0,Casper (CM-113),5758,198.0,20861,-1144.0


## **Comparación de los datos**

Nuestro objetivo es predecir la cantidad de muestras que podria traer cada mision Artemis, pero no tenemos informacion (por ahora, julio 2021) sobre los datos tecnicos de que tipo de vehiculo se va a utilizar. Lo que si tenemos es una hoja con los datos del [Sistema de Lanzamiento Espacial (SLS) y de los modulos Orion](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf), podemos extraer de ahi pesos y las cargas.

Entendemos como "carga"a la cantidad total de peso que un cohete puede transportar al espacio. Obviamente la decision de la carga afecta al diseño de los cohetes y viceversa.

Se sabe que la carga del Saturno V era 43 500 kg y que los pesos de los módulos han variado de una misión a otra. Por tanto, para determinar las proporciones que permitirán realizar predicciones sobre las misiones Artemis, se puede usar lo siguiente:

    Carga de Saturno V
    Peso de las muestras de la misión
    Peso de los módulos de la misión

In [43]:
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff,Crewed area : Payload,Sample : Crewed area,Sample : Payload
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Kitty Hawk (CM-110),5960,0.0,21224,0.0,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Columbia (CSM-107),5609,-351.0,20844,-380.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Challenger (LM-12),16456,1221.0,America (CM-114),5840,231.0,22296,1452.0,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Yankee Clipper (CM-108),5560,-315.0,22005,-300.0,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Intrepid (LM-6),15103,-1342.0,Casper (CM-113),5758,198.0,20861,-1144.0,0.479563,0.005246,0.002516


**Guardado de las proporciones**

Después, se puede usar la función mean() para tomar el promedio de todas esas proporciones entre todas las misiones.

In [44]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.49630268199233724
0.0028946732226251396
0.0014369195019157093


Luego, estas proporciones se pueden usar para predecir la capacidad de Artemis para transportar muestras

## **Predicción de la capacidad de transportar muestras de Artemis**

**Creación de un elemento dataframe para la misión Artemis**

No se tienen todos los detalles sobre la misión Artemis, pero se sabe que, actualmente, en cada misión el cohete realizará ciclos de tres iteraciones (esto es teniendo en cuenta los lanzamientos tradicionales que realizo la NASA en sus misiones Apolo) Cada cohete tendrá una versión tripulada y otra para transporte de mercancías. Para los fines de este módulo, solo se centrará en los tres cohetes diseñados para albergar a la tripulación, y alinearlos más con las misiones Apolo. También se sabe que la carga prevista del Sistema de lanzamiento espacial (SLS) crecerá en cada iteración, pero que el peso actual de Orion (la combinación de los módulos de mando y lunar) tiene un peso estimado en la actualidad.

Una vez más, los módulos de mando y lunar se denominarán Área de la tripulación, y se puede crear un elemento dataframe con la información disponible sobre las tres misiones tripuladas:

In [45]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

,Mission,Total weight (kg),Payload (kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2,26520,42955


Se puede calcular el peso de las muestras en función de las proporciones determinadas a partir de las misiones Artemis:

In [46]:
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample weight from total (kg),Sample weight from payload (kg)
0,artemis1,26520,26988,76.766734,38.779584
1,artemis1b,26520,37965,76.766734,54.552649
2,artemis2,26520,42955,76.766734,61.722877


Por último, se puede obtener el promedio de las dos predicciones:

In [47]:
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (kg)'] + artemis_mission['Sample weight from total (kg)'])/2
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample weight from total (kg),Sample weight from payload (kg),Estimated sample weight (kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis1b,26520,37965,76.766734,54.552649,65.659691
2,artemis2,26520,42955,76.766734,61.722877,69.244806


## **Prioridad de la obtención de muestras de rocas lunares en función de los datos**

El siguiente paso sera tratar de responder que tipo de muestras deberian traer en las misiones Artemis. Ya tenemos los datos previos de lo que nos aportaron las misiones Apolo, con sus respectivos tipos de piedras y cantidad de cada una (kg), tambien tenemos una idea aproximada de la carga que podria traernos Artemis, entonces podriamos saber ahora cuantos Kg de cada tipo de piedra deberia traernos Artemis para tener suficientes muestras sin tener demasiadas muestras repetidas.

In [48]:
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)'] * (rock_samples['Pristine (%)'] * .01)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.0448,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.0534,40.31,0.021526


En este momento, no resulta útil examinar **head()** ni **info()** en el elemento dataframe rock_samples. Con más de 2000 muestras, es complicado comprender cuáles son los valores. Para ello, puede usar la función **describe()**:

In [49]:
rock_samples.describe()

,ID,Weight (kg),Pristine (%),Remaining (kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


Los datos que observamos es que cada muestra pesa (de media) 0,16 Kg y que queda un 84% de la cantidad obtenido por las misiones Apolo, estos valores nos ayudan a seleccionar que tipo de muestras deberia traer Artemis.

In [50]:
low_samples = rock_samples.loc[(rock_samples['Weight (kg)'] >= .16) & (rock_samples['Pristine (%)'] <= 50)]
low_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.973,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.425,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.250,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.185,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.919,35.15,0.323028


Podemos extraer por cantidad cuantas muestras tenemos, eso ayudara a poder hacer una prediccion de que tipos de muestras son las mas "criticas" y a las que se les deberia dar prioridad de extraccion.

In [51]:
low_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight (kg)     27 non-null     float64
 5   Pristine (%)    27 non-null     float64
 6   Remaining (kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


Lo expresado anteriormente nos dice que son 27 las muestras.

Usaremos la funcion **unique()** para ver cuántos tipos únicos hay en los elementos **dataframe low_samples** y **rock_samples**.

Para **dataframe low_samples**:

In [52]:
low_samples.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

Para **rock_samples**:

In [53]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

Se puede ver que, aunque entre todas las muestras se han obtenido seis tipos únicos, aquellas de las que hay menos cantidad solo se corresponden a cuatro tipos únicos. Pero esto no indica todo sobre las muestras en las que se podría centrar. Por ejemplo, en el elemento **dataframe low_samples**, ¿de cuántos de cada uno de los tipos se considera que la cantidad es baja?. Para eso vamos a agrupar las muestra segun tipo de piedra para saber cuantas tenemos de cada una.

In [54]:
low_samples.groupby('Type')['Weight (kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight (kg), dtype: int64

Los datos obtenidos nos muestran que que hay mas piedras de tipo Basalt y Breccia.Además, debido a la elevada probabilidad de que en cada misión haya que obtener muestras de núcleo y tierra, se puede centrar en los tipos de basalto y brecha para las que es necesario recoger:

In [58]:
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight (kg)     22 non-null     float64
 5   Pristine (%)    22 non-null     float64
 6   Remaining (kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


Se puede comparar el peso total del elemento dataframe **needed_samples** con el de **rock_samples**. Es decir, se compararán las muestras identificadas con bajas existencias con todas las obtenidas por las misiones Apolo.

Para las muestras que necesitamos:

In [59]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight (kg), dtype: float64

Para las muestras que ya tenemos:

In [60]:
rock_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight (kg), dtype: float64

Hay un dato que destaca especialmente: nunca ha habido una gran cantidad de piedras tipo Crustal.

Se pueden agregar piedra de ese tipo al conjunto de muestras necesarias

In [61]:
needed_samples = needed_samples.append(rock_samples.loc[rock_samples['Type'] == 'Crustal'])
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              68 non-null     int64  
 1   Mission         68 non-null     object 
 2   Type            68 non-null     object 
 3   Subtype         68 non-null     object 
 4   Weight (kg)     68 non-null     float64
 5   Pristine (%)    68 non-null     float64
 6   Remaining (kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


## **Resumen de las muestras necesarias:**

El paso final consiste en consolidar todo el conocimiento en una tabla que se pueda compartir con los astronautas. En primer lugar, se necesita una columna para cada tipo de roca que ya se ha identificado como del que se quieren obtener más muestras:

In [63]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


A continuación, necesita el peso total de cada tipo de roca obtenida originalmente:

In [64]:
needed_sample_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,4.74469


Cuando los astronautas estén en la Luna, una forma de identificar las rocas es por su tamaño. Si se les puede indicar el tamaño estimado de cada tipo de roca, se podría facilitar el proceso de obtención.

In [65]:
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg),Average weight (kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264812
2,Crustal,4.74469,0.103145


Seguramente quiera indicarles a los astronautas qué cantidad de cada tipo quiere que consigan. Por tanto, para los tres tipos que busca, debería tomar el número total de cada tipo y obtener el porcentaje restante de cada tipo de roca.

In [66]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of samples'].sum()
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Number of samples'] / total_rocks
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of samples,Percentage of rocks
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264812,959,0.707227
2,Crustal,4.74469,0.103145,46,0.033923


Por último, para asociarlo todo a una recomendación para el programa Artemis, se puede determinar el peso medio de las muestras que se han estimado anteriormente.

In [67]:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

64.22588520079607

Este número se puede usar para determinar qué cantidad de cada roca deben intentar conseguir los astronautas:

In [68]:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks'] * artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect'] / needed_samples_overview['Average weight (kg)']
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of samples,Percentage of rocks,Weight to collect,Rocks to collect
0,Basalt,17.42340,1.244529,351,0.258850,16.624842,13.358345
1,Breccia,10.11850,1.264812,959,0.707227,45.422289,35.912271
2,Crustal,4.74469,0.103145,46,0.033923,2.178754,21.123128


Por tanto, es posible que se le indique a los astronautas de la misión Artemis que intenten conseguir **13 piedras de Basalt, 35 de Breccia y 21 Crustal.**